In [ ]:
import numpy as np
from simforest.cluster import SimilarityForestCluster

# Unsupervised tree fitting

Let's say we want to find distance matrix for the 4 vectors. We'll fit single unsupervised tree and then use it to compare the instances.

First, we sample a pair of vectors that are used to define the split. The first one is sampled randomly, then the second one is sampled, making sure that it's different from the first one.

In [ ]:
# Input
X = np.array([[1.0, 0.0, 2.0], [2.0, 0.0, 2.0], [0.0, 3.0, -1.0], [-1.0, -2.5, 3.6]])

# Number of instances in the current tree node
n = X.shape[0]

# Sample index of first vector randomly
first = np.random.randint(0, n)

# Prepare a pool of vectors different from the first one
others = np.where(np.abs(X - X[first]) > 0)[0]

# Choose the second one from the others pool
second = np.random.choice(others)

Then, we calculate the projection of vectors in current node. The projection is defined as dot(X, second) - dot(X, first). One can notice, that the projection is equivalent to dot(X, second - first).

In the next step, the split point is found in unsupervised way by choosing a random split point at the projection line and splitting the vectors into partitions using this threshold.

In [ ]:
# Calculate the projection
similarities = np.dot(X, X[second]-X[first])

# Figure out the range of similarity values
similarities_min = np.min(similarities)
similarities_max = np.max(similarities)

# Find a random threshold in this range
split_point = np.random.uniform(similarities_min, similarities_max, 1)

# Find indexes of vectors that should go to the left and right children of current node
left_indexes = similarities <= split_point
right_indexes = np.invert(left_indexes)

In [ ]:
# Check which vectors went will be used to build left children node
X[left_indexes]

In [ ]:
# Check which vectors went will be used to build right children node
X[right_indexes]

The process of partitioning the dataset continues recursively until the depth limit is reached or all examples in each tree nodes are the same.

# Calculating distance matrix

After the tree is grown, each pair of instances is passed through the tree, and the depth at which the pair splits is used to determine dissimilarity of the instances. The dissimilarity is 1 / depth at which the pair splits. This way, we achieve maximal distance if the pair split in the first node of the tree, and small distance if the pair splits deep down the tree.

The draft of a function calculating the similarity can be found below.

In [ ]:
cdef int similarity(self, float [:] xi, float [:] xj) nogil:
    """Calculate similarity of a pair of data-points in tree-space.
        The pair of traverses down the tree, and the depth on which the pair splits is recorded.
        This values serves as a similarity measure between the pair.
        Parameters
        ----------
        X : array-like matrix of shape = [n_samples, n_features]
            The training data samples.
        Returns
        -------
        int : the depth on which the pair splits.
    """
    if self.is_leaf:
        return self.depth

    cdef bint path_i = self.projection(xi, self._p, self._q) <= self._split_point
    cdef bint path_j = self.projection(xj, self._p, self._q) <= self._split_point

    if path_i == path_j:
        # the same path, check if the pair goes left or right
        if path_i:
            return self._lhs.similarity(xi, xj)
        else:
            return self._rhs.similarity(xi, xj)
    else:
        # different path, return current depth
        return self.depth

Final distance is obtained by averaging the depth at which the pairs of objects plit across all tree. This is implemented in the function below. Note, that the resulting distance matrix is not square (n_objects, n_objects), but one-dimensional, condensed matrix of shape comb(n_objects, 2).

In [ ]:
cpdef np.ndarray[np.float32_t, ndim=1] predict_(self, np.ndarray[np.float32_t, ndim=2] X):
    """Produce pairwise distance matrix.
        Parameters
        ----------
        X : array-like matrix of shape = [n_samples, n_features]
            The training data samples.
        Returns
        -------
        distance_matrix : ndarray of shape = comb(n_samples, 2) containing the distances
    """
    cdef int n = X.shape[0]
    cdef float [:, :] X_view = X
    cdef np.ndarray[np.float32_t, ndim=1] distance_matrix = np.ones(<int>comb(n, 2), np.float32, order='c')
    cdef float [:] view = distance_matrix

    cdef int num_threads = 4
    cdef int diagonal = 1
    cdef int idx = 0
    cdef float similarity = 0.0
    cdef int i = 0
    cdef int j = 0
    cdef int e = 0
    cdef CSimilarityTreeCluster current_tree

    for i in range(n):
        for j in range(diagonal, n):
            for e in range(self.n_estimators,):
                current_tree = self.estimators_[e]
                similarity += current_tree.similarity(X_view[i], X_view[j])

            # similarity is an average depth at which points split across all trees
            # but we don't need to divide the accumulated similarities as this only rescales the distance
            view[idx] = 1 / <float>similarity
            similarity = 0.0
            idx += 1
        diagonal += 1

    return distance_matrix